In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import autograd.numpy as np_   # Thinly-wrapped version of Numpy
from autograd import grad

## Características mais impactantes em casos de AVC

Em busca de obter resultados sobre quais características mais aparecem nos casos de AVC, como grupo, decidimos usar o classificador de Árvore de Decisão com poda nos dados, equilíbrio entre pessoas que tiveram AVC e que não tiveram, para assim obter as tais características mais impactantes. Para melhorar os resultados obtidos, além da poda do DataFrame mencionada acima, também foi realizada a exclusão de índices numéricos, como é o caso da idade, índice de massa corporal (IMC) e nível de glicose. Após tudo isso, realizamos o teste 10.000 vezes para, então, obter as top 5 características mais impactantes, apresentadas na tabela abaixo:

| Category                | Count |
|-------------------------|-------|
| hypertension             | 6468  |
| work_type_children       | 4330  |
| heart_disease            | 3655  |
| work_type_Private        | 3561  |
| smoking_status_smokes    | 3273  |

In [19]:
df = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
df = df[df['bmi'].notna()]
df_p = pd.concat([df[df['stroke'] == 0].head(len(df[df['stroke'] == 1])), df[df['stroke'] == 1]])
df_features = df_p[['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']]
df_features = pd.get_dummies(df_features)
df_outcome = df_p[['stroke']].replace(0,-1)
df_outcome = pd.get_dummies(df_outcome)

most_occuring = list()
for i in range (10000):
    X_train, X_test, y_train, y_test = train_test_split(df_features, df_outcome, train_size=0.5)
    X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

    tree = DecisionTreeClassifier(criterion='entropy')
    tree = tree.fit(X_train, y_train)

    y_estimado = tree.predict(X_test)

    importances = tree.feature_importances_
    importances_indices = np.argsort(importances)[::-1]
    importances_indices = importances_indices[:5]

    colunas = list(df_features.head(0))

    for i in range(5):
        most_occuring.append(colunas[importances_indices[i]])

most_occuring = pd.DataFrame(most_occuring)
most_occuring = most_occuring.value_counts()
most_occuring.head(5)

hypertension             6468
work_type_children       4330
heart_disease            3655
work_type_Private        3561
smoking_status_smokes    3273
dtype: int64